In [1]:
import QuantumToolbox as qt
using Revise
using LinearAlgebra
import SciMLBase: solve
import OrdinaryDiffEq as ode
include("/Users/gavinrockwood/Documents/Git Repositories/SuperconductingCavities/src_qt/SuperconductingCavities.jl")

using Logging
using MiniLoggers

In [2]:
# MiniLogger(minlevel = MiniLoggers.Info) |> global_logger
InfoLogger = MiniLogger(minlevel = MiniLoggers.Info)
ProgressLogger = MiniLogger(minlevel = LogLevel(-1))
DebugLogger = MiniLogger(minlevel = MiniLoggers.Debug)

global_logger(DebugLogger)

Base.CoreLogging.SimpleLogger(VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(21) open, 0 bytes waiting)), VSCodeServer.io_send_callback), Info, Dict{Any, Int64}())

In [7]:
Eᶜ = 0.10283303447280807;
Eʲ = 26.96976142643705;
Eᵒˢᶜ = [6.2280871213];
g = [0.026184375000000013];
ng  =  0;
N=8;
N_cut = 60;

In [8]:
test = SC_qt.Transmon_Resonators_Constructor(Eᶜ, Eʲ, Eᵒˢᶜ, g, N, [N]);

In [47]:
ψi = test.dressed_states[(0,0)]
ν = test.dressed_energies[(1,0)] - test.dressed_energies[(0,0)]
ε = 0.005
#Ĥₜ = SC_qt.Get_Drive_Hamiltonian(test, test.n̂ₜ, ν, 0.003)

op = qt.dense_to_sparse(test.n̂ₜ)
function cf(t)
    return 2*pi*(ε*sin(2π*ν*t))
end
Lₜ = qt.TimeDependentOperatorSum([cf], [qt.liouvillian(op)]);


Ĥₜ= qt.TimeDependentOperatorSum([cf], [op]);

In [90]:
mdm = qt.maximally_mixed_dm(test.Ĥ.dims)

Quantum Object:   type=Operator   dims=[8, 8]   size=(64, 64)   ishermitian=true
64×64 Diagonal{ComplexF64, Vector{ComplexF64}}:
 0.015625-0.0im           ⋅      …           ⋅               ⋅    
          ⋅      0.015625-0.0im              ⋅               ⋅    
          ⋅               ⋅                  ⋅               ⋅    
          ⋅               ⋅                  ⋅               ⋅    
          ⋅               ⋅                  ⋅               ⋅    
          ⋅               ⋅      …           ⋅               ⋅    
          ⋅               ⋅                  ⋅               ⋅    
          ⋅               ⋅                  ⋅               ⋅    
          ⋅               ⋅                  ⋅               ⋅    
          ⋅               ⋅                  ⋅               ⋅    
         ⋮                       ⋱                  
          ⋅               ⋅      …           ⋅               ⋅    
          ⋅               ⋅                  ⋅               ⋅    
          ⋅   

In [93]:
tlist = collect(LinRange(0.0, Real(1/ν), 100));
I = qt.eye(64, dims = test.Ĥ.dims)
L = qt.liouvillian(test.Ĥ)
meres = qt.mesolve(L, mdm, tlist, H_t = Lₜ)

Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)


Solution of time evolution
(return code: Success)
--------------------------
num_states = 100
num_expect = 0
ODE alg.: OrdinaryDiffEqTsit5.Tsit5{typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!), Static.False}(OrdinaryDiffEqCore.trivial_limiter!, OrdinaryDiffEqCore.trivial_limiter!, static(false))
abstol = 1.0e-8
reltol = 1.0e-6


In [95]:
meres.states[end]

Quantum Object:   type=Operator   dims=[8, 8]   size=(64, 64)   ishermitian=true
64×64 Matrix{ComplexF64}:
 0.015625+0.0im       0.0+0.0im  …       0.0+0.0im       0.0+0.0im
      0.0+0.0im  0.015625+0.0im          0.0+0.0im       0.0+0.0im
      0.0+0.0im       0.0+0.0im          0.0+0.0im       0.0+0.0im
      0.0+0.0im       0.0+0.0im          0.0+0.0im       0.0+0.0im
      0.0+0.0im       0.0+0.0im          0.0+0.0im       0.0+0.0im
      0.0+0.0im       0.0+0.0im  …       0.0+0.0im       0.0+0.0im
      0.0+0.0im       0.0+0.0im          0.0+0.0im       0.0+0.0im
      0.0+0.0im       0.0+0.0im          0.0+0.0im       0.0+0.0im
      0.0+0.0im       0.0+0.0im          0.0+0.0im       0.0+0.0im
      0.0+0.0im       0.0+0.0im          0.0+0.0im       0.0+0.0im
         ⋮                       ⋱                  
      0.0+0.0im       0.0+0.0im  …       0.0+0.0im       0.0+0.0im
      0.0+0.0im       0.0+0.0im          0.0+0.0im       0.0+0.0im
      0.0+0.0im       0.0+0.0im     

In [75]:
seres = qt.sesolve(test.Ĥ, ψi, tlist)

Progress: [==============================] 100.0% --- Elapsed Time: 0h 00m 00s (ETA: 0h 00m 00s)


Solution of time evolution
(return code: Success)
--------------------------
num_states = 101
num_expect = 0
ODE alg.: OrdinaryDiffEqTsit5.Tsit5{typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!), Static.False}(OrdinaryDiffEqCore.trivial_limiter!, OrdinaryDiffEqCore.trivial_limiter!, static(false))
abstol = 1.0e-8
reltol = 1.0e-6


In [57]:
a = meres.states[end]
b = seres.states[end]

b'*a*b

0.9992740896791503 + 1.6653345369377348e-16im

Union{Nothing, Function}

In [30]:
drive_coef = nothing

drive_coef = (drive_coef isa Nothing) ? t->1 : drive_coef


#11 (generic function with 1 method)

In [53]:
function testf(t= nothing)
    print("hi")
end

testf (generic function with 4 methods)

In [54]:
testf(1)

hi